In [1]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import shared

ve_or_ju = "ve"

In [2]:
running_order = pd.read_csv(f'data/running_order_j2019_{ve_or_ju}.tsv', delimiter="\t")
display(running_order.shape)

(5086, 7)

In [3]:
running_order["leg_nro"] = running_order["leg"]
running_order["orig_name"] = running_order["name"]
running_order["name"] = running_order["name"].str.lower()

In [4]:
running_order.head()

,team_id,team,team_base_name,team_country,leg,leg_dist,name,leg_nro,orig_name
0,3,MS Parma 1,MS Parma,FIN,1,6.0,anastasia rudnaya,1,Anastasia Rudnaya
1,3,MS Parma 1,MS Parma,FIN,2,5.7,anni heikkonen,2,Anni Heikkonen
2,3,MS Parma 1,MS Parma,FIN,3,7.3,tiia istolahti,3,Tiia Istolahti
3,3,MS Parma 1,MS Parma,FIN,4,7.9,maija sianoja,4,Maija Sianoja
4,8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,SWE,1,6.0,fiona bunn,1,Fiona Bunn


In [5]:
predictions_and_history = pd.read_csv(f"data/preds_for_runners_with_history_2_{ve_or_ju}.csv", delimiter="\t")
display(predictions_and_history.shape)
predictions_and_history.head()


(13831, 13)

,Unnamed: 0,mean_team_id,num_valid_times,mean_pace,stdev,prior_mean,prior_log_std,predicted_pace,pred_pace_error,predicted_log_pace_mean,predicted_log_pace_std,name,teams
0,0,7.0,3,6.649,0.404,7.002,0.102,6.981,0.332,1.943,0.099,anna mårsell,STORA TUNA OK
1,1,27.5,6,7.096,0.609,6.540,0.124,6.658,0.438,1.896,0.108,magdalena olsson,IFK MORAS OK;STORA TUNA OK
2,2,3.0,1,5.944,0.000,6.871,0.137,6.838,0.894,1.922,0.138,julia jakob,STORA TUNA OK
3,3,8.5,6,6.077,0.362,6.624,0.129,6.470,0.393,1.867,0.111,tove alexandersson,STORA TUNA OK
4,4,61.0,3,7.328,0.845,7.742,0.115,7.704,0.376,2.042,0.111,lisa holer,EKSJÖ SOK;GÖTEBORG MAJORNA OK


In [6]:

predictions_and_history["num_runs"] = predictions_and_history["num_valid_times"]
no_history_row = pd.DataFrame([[0, 0, 0]], columns=["predicted_log_pace_mean", "predicted_log_pace_std", "num_valid_times"])
def get_history_and_preds(running_order_row):
    history_row = shared.get_matching_history_row_for_runner(running_order_row, predictions_and_history, no_history_row)
    #print(f"estimate_row log_means {history_row.log_means} {history_row.log_stdevs}")
    pred_log_mean = history_row.predicted_log_pace_mean.values[0]
    pred_log_std = history_row.predicted_log_pace_std.values[0]
    num_valid_times = history_row.num_valid_times.values[0]
    return pd.Series({"pred_log_mean": pred_log_mean, "pred_log_std": pred_log_std, "num_valid_times": num_valid_times})

history_and_preds = running_order.apply(lambda row: get_history_and_preds(row), axis=1)
running_order = running_order.assign(num_runs = history_and_preds.num_valid_times)
running_order = running_order.assign(pred_log_mean = history_and_preds.pred_log_mean)
running_order = running_order.assign(pred_log_std = history_and_preds.pred_log_std)



name 'sanna timonen' team_name 'MAASOTAKOULU LPR'
by_name 0 by_name_and_colon 5 runners 2
Duplicate runner       Unnamed: 0  mean_team_id  num_valid_times  mean_pace  stdev  prior_mean  \
7072        7350         501.0                1     13.403    0.0      10.767   
7075        7353         777.0                1     10.631    0.0      12.799   

      prior_log_std  predicted_pace  pred_pace_error  predicted_log_pace_mean  \
7072          0.153          10.832            2.571                    2.383   
7075          0.203          12.684            2.053                    2.540   

      predicted_log_pace_std                                   name  \
7072                   0.151         sanna timonen:MAASOTAKOULU LPR   
7075                   0.201  sanna timonen:MAASOTAKOULU LPR VENLAT   

                        teams  num_runs  
7072         MAASOTAKOULU LPR         1  
7075  MAASOTAKOULU LPR VENLAT         1  
name 'sanna valkonen' team_name 'KAUKAAN LYLY'
by_name 0 by_name_

In [7]:

(top_countries, top_first_names) = shared.read_persisted_dummy_column_values(ve_or_ju)

In [8]:
features = shared.preprocess_features(running_order, top_countries, ve_or_ju)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5086 entries, 0 to 5085
Data columns (total 12 columns):
team_id           5086 non-null int64
team              5086 non-null object
team_base_name    5086 non-null object
team_country      5086 non-null object
leg               5086 non-null int64
leg_dist          5086 non-null float64
name              5086 non-null object
leg_nro           5086 non-null int64
orig_name         5086 non-null object
num_runs          5086 non-null float64
pred_log_mean     5086 non-null float64
pred_log_std      5086 non-null float64
dtypes: float64(4), int64(3), object(5)
memory usage: 476.9+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 931 entries, 0 to 930
Data columns (total 3 columns):
first_name           930 non-null object
fn_pace_class        931 non-null int64
fn_pace_std_class    931 non-null int64
dtypes: int64(2), object(1)
memory usage: 21.9+ KB


None

[]

In [9]:
features.info()

<class 'pandas.core.sparse.frame.SparseDataFrame'>
RangeIndex: 5086 entries, 0 to 5085
Data columns (total 40 columns):
team_id                  5086 non-null Sparse[int64, nan]
team_id_log10            5086 non-null Sparse[float64, nan]
team_id_square           5086 non-null Sparse[int64, nan]
leg_1                    5086 non-null Sparse[uint8, 0]
leg_2                    5086 non-null Sparse[uint8, 0]
leg_3                    5086 non-null Sparse[uint8, 0]
leg_4                    5086 non-null Sparse[uint8, 0]
c_EST                    5086 non-null Sparse[uint8, 0]
c_FIN                    5086 non-null Sparse[uint8, 0]
c_GBR                    5086 non-null Sparse[uint8, 0]
c_LAT                    5086 non-null Sparse[uint8, 0]
c_NOR                    5086 non-null Sparse[uint8, 0]
c_OTHER                  5086 non-null Sparse[uint8, 0]
c_RUS                    5086 non-null Sparse[uint8, 0]
c_SUI                    5086 non-null Sparse[uint8, 0]
c_SWE                    5086 no

In [10]:

#gbr_sd_estimate = shared.predict_without_history(features)
gbr_sd_estimate = shared.predict_without_history(features, ve_or_ju)

,log_q_low,predicted,log_q_high,log_std
0,1.875,6.891,2.003,0.064
1,1.808,7.057,2.008,0.100
2,1.851,6.850,2.108,0.128
3,1.870,6.408,2.000,0.065
4,1.864,6.744,2.034,0.085
5,1.809,6.883,2.054,0.122
6,1.727,6.108,2.057,0.165
7,1.845,7.475,2.090,0.122
8,1.872,7.463,2.050,0.089
9,1.858,7.234,2.073,0.108


0.1683543829131468

In [11]:
running_order["predicted"] = gbr_sd_estimate["predicted"]
running_order["log_q_low"] = gbr_sd_estimate["log_q_low"]
running_order["log_q_high"] = gbr_sd_estimate["log_q_high"]
running_order["log_std"] = gbr_sd_estimate["log_std"]


In [12]:
running_order.head()

,team_id,team,team_base_name,team_country,leg,leg_dist,name,leg_nro,orig_name,num_runs,pred_log_mean,pred_log_std,predicted,log_q_low,log_q_high,log_std
0,3,MS Parma 1,MS Parma,FIN,1,6.0,anastasia rudnaya,1,Anastasia Rudnaya,4.0,1.928,0.096,6.891132,1.875490,2.002510,0.063510
1,3,MS Parma 1,MS Parma,FIN,2,5.7,anni heikkonen,2,Anni Heikkonen,6.0,2.046,0.098,7.056842,1.808388,2.007626,0.099619
2,3,MS Parma 1,MS Parma,FIN,3,7.3,tiia istolahti,3,Tiia Istolahti,1.0,1.916,0.109,6.849927,1.851113,2.107522,0.128204
3,3,MS Parma 1,MS Parma,FIN,4,7.9,maija sianoja,4,Maija Sianoja,7.0,1.950,0.086,6.407751,1.869919,2.000480,0.065281
4,8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,SWE,1,6.0,fiona bunn,1,Fiona Bunn,1.0,2.110,0.172,6.743671,1.863840,2.033659,0.084909


In [13]:
running_order["log_std"].describe(percentiles=[0.01, 0.05, .25, .5, .75, .95, .99])

count    5086.000000
mean        0.168354
std         0.049119
min        -0.057098
1%          0.010700
5%          0.046180
25%         0.149235
50%         0.175321
75%         0.198421
95%         0.231501
99%         0.254085
max         0.319118
Name: log_std, dtype: float64

In [14]:
running_order["log_std_fixed"] = np.clip(running_order["log_std"], 0.1, 0.5)
#running_order["log_std"].values[running_order["log_std"].values < 0] = 0.1

In [15]:
#def select_final_ind_preds(row):
#    return pd.Series({"pred_log_mean": pred_log_mean, "pred_log_std": pred_log_std, "num_valid_times": num_valid_times})
    
    
#final_ind_preds = running_order.apply(lambda row: select_final_ind_preds(row), axis=1)

running_order["final_pace_mean"] = np.log(running_order["predicted"])
running_order["final_pace_std"] = running_order["log_std_fixed"]
use_predicted_mean = running_order["num_runs"].values >= 1
running_order["final_pace_mean"].values[use_predicted_mean] = running_order["pred_log_mean"].values[use_predicted_mean]
use_predicted_std = running_order["num_runs"].values >= 3
running_order["final_pace_std"].values[use_predicted_std] = running_order["pred_log_std"].values[use_predicted_std]
running_order.head().round(3)



,team_id,team,team_base_name,team_country,leg,leg_dist,name,leg_nro,orig_name,num_runs,pred_log_mean,pred_log_std,predicted,log_q_low,log_q_high,log_std,log_std_fixed,final_pace_mean,final_pace_std
0,3,MS Parma 1,MS Parma,FIN,1,6.0,anastasia rudnaya,1,Anastasia Rudnaya,4.0,1.928,0.096,6.891,1.875,2.003,0.064,0.100,1.928,0.096
1,3,MS Parma 1,MS Parma,FIN,2,5.7,anni heikkonen,2,Anni Heikkonen,6.0,2.046,0.098,7.057,1.808,2.008,0.100,0.100,2.046,0.098
2,3,MS Parma 1,MS Parma,FIN,3,7.3,tiia istolahti,3,Tiia Istolahti,1.0,1.916,0.109,6.850,1.851,2.108,0.128,0.128,1.916,0.128
3,3,MS Parma 1,MS Parma,FIN,4,7.9,maija sianoja,4,Maija Sianoja,7.0,1.950,0.086,6.408,1.870,2.000,0.065,0.100,1.950,0.086
4,8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,SWE,1,6.0,fiona bunn,1,Fiona Bunn,1.0,2.110,0.172,6.744,1.864,2.034,0.085,0.100,2.110,0.100


In [16]:
#running_order.tail(15).round(3)

In [17]:
running_order.to_csv(f"data/running_order_2019_with_estimates_{ve_or_ju}.tsv", "\t")

In [18]:
running_order[running_order['team'].str.contains("Reak")].round(2)

,team_id,team,team_base_name,team_country,leg,leg_dist,name,leg_nro,orig_name,num_runs,pred_log_mean,pred_log_std,predicted,log_q_low,log_q_high,log_std,log_std_fixed,final_pace_mean,final_pace_std


In [19]:
running_order[
    ['num_runs', 'pred_log_mean', "pred_log_std", "predicted", "log_std_fixed", "final_pace_mean", "final_pace_std"]
].groupby('num_runs').agg(["mean"]).round(2)



,pred_log_mean,pred_log_std,predicted,log_std_fixed,final_pace_mean,final_pace_std
,mean,mean,mean,mean,mean,mean
num_runs,,,,,,
0.0,0.00,0.00,12.64,0.17,2.52,0.17
1.0,2.50,0.18,12.59,0.18,2.50,0.18
2.0,2.49,0.18,12.24,0.18,2.49,0.18
3.0,2.45,0.17,12.01,0.17,2.45,0.17
4.0,2.42,0.15,11.74,0.17,2.42,0.15
5.0,2.40,0.14,11.61,0.17,2.40,0.14
6.0,2.36,0.13,11.04,0.17,2.36,0.13
7.0,2.37,0.12,11.13,0.16,2.37,0.12
